In [12]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re 
import urllib
import time

In [13]:
options = Options()
options.headless = True
driver = webdriver.Chrome('./chromedriver',options=options)

In [14]:
#uses webdriver object to execute javascript code and get dynamically loaded webcontent
def get_js_soup(url,driver):
    driver.get(url)
    res_html = driver.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

In [15]:
def scrape_dir_page(dir_url,driver):
    print ('-'*20,'Scraping directory page','-'*20)
    new_links = []
    new_base_url = 'https://www.nbcnews.com/'
    soup = get_js_soup(dir_url,driver)
    for link_holder in soup.find_all('li',class_='alacarte__item'):
        rel_link = link_holder.find('a')['href']  #get url
        if rel_link.startswith('https://www.nbcnews.com/video/') or not rel_link.startswith('https://www.nbcnews.com/'):
            continue
        new_links.append(rel_link)
    print ('-'*20,'Found {} news urls'.format(len(new_links)),'-'*20)
    return new_links

In [16]:
dir_url = 'https://www.nbcnews.com/'
news_links = scrape_dir_page(dir_url,driver)

-------------------- Scraping directory page --------------------
-------------------- Found 36 news urls --------------------


In [17]:
def scrape_news(news_url,driver):
    soup = get_js_soup(news_url,driver)   
    title = soup.select("h1")[0]
    
    content = soup.find_all(class_="article-body__content")
    content_str = ""
    for p in content:
        content_str += p.text
    return title.contents[0], content_str

In [18]:
output = {}
tot_urls = len(news_links)
for i,link in enumerate(news_links):
    print ('-'*20,'Scraping news url {}/{}'.format(i+1,tot_urls),'-'*20)
    title,content = scrape_news(link,driver)
    output[title] = (content, link)

-------------------- Scraping news url 1/36 --------------------
-------------------- Scraping news url 2/36 --------------------
-------------------- Scraping news url 3/36 --------------------
-------------------- Scraping news url 4/36 --------------------
-------------------- Scraping news url 5/36 --------------------
-------------------- Scraping news url 6/36 --------------------
-------------------- Scraping news url 7/36 --------------------
-------------------- Scraping news url 8/36 --------------------
-------------------- Scraping news url 9/36 --------------------
-------------------- Scraping news url 10/36 --------------------
-------------------- Scraping news url 11/36 --------------------
-------------------- Scraping news url 12/36 --------------------
-------------------- Scraping news url 13/36 --------------------
-------------------- Scraping news url 14/36 --------------------
-------------------- Scraping news url 15/36 --------------------
-------------------

In [19]:
def write_url(lst,file_):
    with open(file_,'w') as f:
        for key, value in lst.items():
            f.write(key + ". " + value[1] + "\n")
news_urls_file = 'nbc#1121urls.txt'
write_url(output,news_urls_file)

In [20]:
def write_content(lst,file_):
    with open(file_,'w') as f:
        for key, value in lst.items():
            f.write(key + ". " + value[0] + "\n")
news_contents_file = 'nbc#1121.txt'
write_content(output,news_contents_file)